In [1]:
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

In [2]:
df = yf.download('GC=F NVDA IONQ', start='2021-01-04')

[*********************100%***********************]  3 of 3 completed


In [3]:
returns = df['Close'].pct_change().dropna().values

C:\Users\User\AppData\Local\Temp\ipykernel_31684\3412915546.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df['Close'].pct_change().dropna().values


In [4]:
cov_matrix = np.cov(returns, rowvar=False)
cov_matrix

array([[8.35007970e-05, 3.48589543e-05, 1.41473726e-05],
       [3.48589543e-05, 3.26223118e-03, 7.26937504e-04],
       [1.41473726e-05, 7.26937504e-04, 1.12668070e-03]])

In [5]:
expected_returns = np.mean(returns, axis=0)

In [6]:
# Eigen decomposition
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# Shrinkage: Replace small eigenvalues with the mean
mean_eigenvalue = np.mean(eigenvalues)
shrunk_eigenvalues = np.maximum(eigenvalues, mean_eigenvalue)

# Reconstruct the covariance matrix
rie_cov_matrix = eigenvectors @ np.diag(shrunk_eigenvalues) @ eigenvectors.T

In [7]:
num_assets = cov_matrix.shape[0]

In [8]:
# Define the optimization problem
def portfolio_variance(weights):
    return weights.T @ cov_matrix @ weights

# Constraints: weights sum to 1
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

# Bounds: weights between 0 and 1
bounds = [(0, 1) for _ in range(num_assets)]

# Initial guess: equal allocation
initial_guess = np.ones(num_assets) / num_assets

In [9]:
# Solve the optimization problem
result = minimize(portfolio_variance, initial_guess, bounds=bounds, constraints=constraints)

In [10]:
result.x

array([6.44867846e-01, 2.77555756e-17, 3.55132154e-01])

In [14]:
target_return = (expected_returns.min() + expected_returns.max()) / 2

In [16]:
risk_aversion=0.1

In [17]:
# Objective function: minimize risk-adjusted return
def objective_function(weights):
    portfolio_variance = weights.T @ cov_matrix @ weights
    portfolio_return = expected_returns @ weights
    return risk_aversion * portfolio_variance - portfolio_return

# Constraints
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]  # weights sum to 1
if target_return is not None:
    constraints.append({'type': 'ineq', 'fun': lambda w: expected_returns @ w - target_return})

# Bounds: weights between 0 and 1
bounds = [(0, 1) for _ in range(num_assets)]

# Initial guess: equal allocation
initial_guess = np.ones(num_assets) / num_assets

# Solve the optimization problem
result = minimize(objective_function, initial_guess, bounds=bounds, constraints=constraints)

In [18]:
result.x

array([0.        , 0.42277697, 0.57722303])